In [ ]:
## Python course for data scientists in a moderate level, supported by Wagatsuma Lab@Kyutech 
#
# The MIT License (MIT): Copyright (c) 2022 Hiroaki Wagatsuma and Wagatsuma Lab@Kyutech
# 
# Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:
# The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE. */
#
# # @Time    : 2022-03-27 
# # @Author  : Hiroaki Wagatsuma
# # @Site    : https://github.com/hirowgit/2B3_python_owl_logic_database_course
# # @IDE     : Python 3.9.7 (default, Oct 17 2021, 16:26:06) [Clang 10.0.0 (clang-1000.11.45.5)] on darwin
# # @File    : lec1_step4.py 

In [2]:
import requests
import xml.etree.ElementTree as xmlet
import re
import numpy as np
import sys
import os

fname_tag="mpd02"
fext=".txt"

#pubmed URL
url_header="https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
url_footer=["esearch.fcgi","efetch.fcgi"]

#backup data
fout_name="fout_mpd_fullLen01.txt"

fkeys=["marmoset","parkinson","disease"]
conj=" AND "
kterm=fkeys[0]
for label in fkeys[1:len(fkeys)]:
    kterm=kterm+conj+label
    print(kterm)

# search
pubmed_search_url = url_header+url_footer[0]
query_keys = {
    "db":"pubmed",
#     "term":'1-methyl-4-phenyl-1,2,3,6-tetrahydropyridine (MPTP)-lesioned common marmoset used',
    "term":'common marmoset',
    "usehistory":"y"
    # keeping data for the next fetch action
}
print(query_keys)
resDf = requests.get(pubmed_search_url, params=query_keys)
xmlD = resDf.text.encode("utf-8")
targetD = xmlet.fromstring(xmlD)
# print(resDf)
# print(xmlD)

# dig and get from target ID obtained the previous action
pubmed_fetch_url = url_header+url_footer[1]
query_keys = {
    "db" : "pubmed",
    "query_key" : targetD.findtext("QueryKey"),
    "WebEnv" : targetD.findtext("WebEnv"),
    "retmode" : "text",
    "rettype" : "abstract"
}

resDg = requests.get(pubmed_fetch_url, params=query_keys)
with open(fout_name, "w", encoding="utf-8") as fid:
    fid.write(resDg.text)
    
resDg = requests.get(pubmed_fetch_url, params=query_keys)
strAnalysis=resDg.text

pat_header ='<eSearchResult><Count>'
pat_body ='.*(\d+)'
pat_footer ='</Count>'
patternRe =pat_header+pat_body+pat_footer

resOri = re.search(patternRe, resDf.text)
resNumStr=resOri.group()
resNum=int(resNumStr[len(pat_header):-len(pat_footer)])
print(resNum)

blockPos=[]
patternRep="\n(\d+). "
res1st =re.search(patternRep, strAnalysis)
# print(res1st.group())
# print(res1st)
Mstart, Mend = res1st.span()
blockPos.append([Mstart,Mend])
# print([Mstart,Mend])
    
patternRep="\n\n(\d+). "
res_iter =re.finditer(patternRep, strAnalysis)
for res in res_iter:
#     print(res.group())
#     print(res)
    Mstart, Mend = res.span()
    blockPos.append([Mstart,Mend])
#     print([Mstart,Mend])

# len(blockPos)

npblockPos=np.array(blockPos)
blockST=npblockPos[:,0].tolist()
npblockPos[-1,-1]

npblockED=npblockPos[1:npblockPos.shape[0],0]+1
blockED=npblockED.tolist()
blockED.append(len(strAnalysis))

# print(blockED)

if not os.path.isdir(fname_tag):
        os.makedirs(fname_tag)

i=0
k=0
# for i in range(len(blockST)):
while i<len(blockST):
    fout_name=os.path.join(fname_tag,str(k+1)+fext)
    patternRep="\n\n(\d+). "
#     strAnalysis=resDg.text[blockST[i]:blockED[i]]
#     res =re.search(patternRep, strAnalysis)
#     numKey=int(res.group()[2:-2])
#     print(numKey)
    
#     k=581
#     i=581
#     strAnalysis=resDg.text[blockST[i]:blockED[i]]
#     print(strAnalysis)
#     patternRep="\n\n(\d+). "
    strAnalysis=resDg.text[blockST[i]:blockED[i]]
    res =re.search(patternRep, strAnalysis)
    numKey0=int(res.group()[2:-2])
    print(numKey0)
    pre_i=i
    i=i+1
    strAnalysis=resDg.text[blockST[i]:blockED[i]]
    res =re.search(patternRep, strAnalysis)
    numKey1=int(res.group()[2:-2])

    if k==numKey0 and  k+1==numKey1:
        print('OK')
    else:
        while k+1==numKey1:
            strAnalysis=resDg.text[blockST[i]:blockED[i]]
            res =re.search(patternRep, strAnalysis)
            numKey1=int(res.group()[2:-2])
            i=i+1
        print('Corrected')

    strAnalysis=resDg.text[blockST[pre_i]:blockED[i]]
    print(k)
    print(strAnalysis[0:10])
    k=k+1
    i=i+1
    
    
#     if i>len(blockST):
#         break
#     print(res.group())
#     with open(fout_name, "w", encoding="utf-8") as fid:
#         fid.write(resDg.text[blockST[i]:blockED[i]])

marmoset AND parkinson
marmoset AND parkinson AND disease
{'db': 'pubmed', 'term': 'common marmoset', 'usehistory': 'y'}
5023


AttributeError: 'NoneType' object has no attribute 'group'

In [27]:
i=581
strAnalysis=resDg.text[blockST[i]:blockED[i]]
print(strAnalysis)
patternRep="\n\n(\d+). "
strAnalysis=resDg.text[blockST[i]:blockED[i]]
res =re.search(patternRep, strAnalysis)
numKey=int(res.group()[2:-2])
print(numKey)

if i+1==numKey:
    print('eq')
else:
    print('neq')

i=582
strAnalysis=resDg.text[blockST[i]:blockED[i]]
print(strAnalysis)




582. Xenobiotica. 2019 Nov;49(11):1323-1331. doi: 10.1080/00498254.2018.1564401. Epub 
2019 Jun 14.

Regioselective hydroxylation of an antiarrhythmic drug, propafenone, mediated by 
rat liver cytochrome P450 2D2 differs from that catalyzed by human P450 2D6.

Uehara S(1), Murayama N(2), Yamazaki H(2), Suemizu H(1).

Author information: 
(1)a Central Institute for Experimental Animals , Kawasaki , Japan.
(2)b Laboratory of Drug Metabolism and Pharmacokinetics, Showa Pharmaceutical
University , Machida , Japan.

582
eq


1. Propafenone, an antiarrhythmic drug, is a typical human cytochrome P450 (P450)
2D6 substrate used in preclinical studies. Here, propafenone oxidation by
mammalian liver microsomes was investigated in vitro. 2. Liver microsomes from
humans and marmosets preferentially mediated propafenone 5-hydroxylation,
minipig, rat and mouse livers primarily mediated 4'-hydroxylation, but cynomolgus
monkey and dog liver microsomes differently mediated N-despropylation. 3.
Quinine

In [34]:
k=581
i=581
strAnalysis=resDg.text[blockST[i]:blockED[i]]
print(strAnalysis)
patternRep="\n\n(\d+). "
strAnalysis=resDg.text[blockST[i]:blockED[i]]
res =re.search(patternRep, strAnalysis)
numKey0=int(res.group()[2:-2])
print(numKey0)
pre_i=i
i=i+1
strAnalysis=resDg.text[blockST[i]:blockED[i]]
res =re.search(patternRep, strAnalysis)
numKey1=int(res.group()[2:-2])

if k==numKey0 and  k+1==numKey1:
    print('OK')
else:
    while k+1==numKey1:
        strAnalysis=resDg.text[blockST[i]:blockED[i]]
        res =re.search(patternRep, strAnalysis)
        numKey1=int(res.group()[2:-2])
        i=i+1
    print('Corrected')
    
strAnalysis=resDg.text[blockST[pre_i]:blockED[i]]
print(strAnalysis)
k=k+1
i=i+1

strAnalysis=resDg.text[blockST[i]:blockED[i]]
print(strAnalysis)
patternRep="\n\n(\d+). "
strAnalysis=resDg.text[blockST[i]:blockED[i]]
res =re.search(patternRep, strAnalysis)
numKey0=int(res.group()[2:-2])
print(numKey0)
pre_i=i
i=i+1
strAnalysis=resDg.text[blockST[i]:blockED[i]]
res =re.search(patternRep, strAnalysis)
numKey1=int(res.group()[2:-2])

if k==numKey0 and  k+1==numKey1:
    print('OK')
else:
    while k+1==numKey1:
        strAnalysis=resDg.text[blockST[i]:blockED[i]]
        res =re.search(patternRep, strAnalysis)
        numKey1=int(res.group()[2:-2])
        i=i+1
    print('Corrected')
    
strAnalysis=resDg.text[blockST[pre_i]:blockED[i]]
print(strAnalysis)
k=k+1
# i=582
# strAnalysis=resDg.text[blockST[i]:blockED[i]]
# print(strAnalysis)




582. Xenobiotica. 2019 Nov;49(11):1323-1331. doi: 10.1080/00498254.2018.1564401. Epub 
2019 Jun 14.

Regioselective hydroxylation of an antiarrhythmic drug, propafenone, mediated by 
rat liver cytochrome P450 2D2 differs from that catalyzed by human P450 2D6.

Uehara S(1), Murayama N(2), Yamazaki H(2), Suemizu H(1).

Author information: 
(1)a Central Institute for Experimental Animals , Kawasaki , Japan.
(2)b Laboratory of Drug Metabolism and Pharmacokinetics, Showa Pharmaceutical
University , Machida , Japan.

582
Corrected


582. Xenobiotica. 2019 Nov;49(11):1323-1331. doi: 10.1080/00498254.2018.1564401. Epub 
2019 Jun 14.

Regioselective hydroxylation of an antiarrhythmic drug, propafenone, mediated by 
rat liver cytochrome P450 2D2 differs from that catalyzed by human P450 2D6.

Uehara S(1), Murayama N(2), Yamazaki H(2), Suemizu H(1).

Author information: 
(1)a Central Institute for Experimental Animals , Kawasaki , Japan.
(2)b Laboratory of Drug Metabolism and Pharmacokinetics, 

In [73]:
i=0
k=0
# for i in range(len(blockST)):
while i<len(blockST):
# while i<2:
    fout_name=os.path.join(fname_tag,str(k+1)+fext)
    patternRep="\n\n(\d+). "
    if i==0:
        patternRep="\n(\d+). "
    else:
        patternRep="\n\n(\d+). "  
#     strAnalysis=resDg.text[blockST[i]:blockED[i]]
#     res =re.search(patternRep, strAnalysis)
#     numKey=int(res.group()[2:-2])
#     print(numKey)
    
#     k=581
#     i=581
#     strAnalysis=resDg.text[blockST[i]:blockED[i]]
#     print(strAnalysis)
#     patternRep="\n\n(\d+). "
    strAnalysis=resDg.text[blockST[i]:blockED[i]]
    res =re.search(patternRep, strAnalysis)
    if i==0:
        hpos=1
    else:
        hpos=2 
    numKey0=int(res.group()[hpos:-2])
#     print(numKey0)
    pre_i=i
    i=i+1
    patternRep="\n\n(\d+). "
    strAnalysis=resDg.text[blockST[i]:blockED[i]]
    res =re.search(patternRep, strAnalysis)
    numKey1=int(res.group()[2:-2])

#     if k==numKey0 and  k+1==numKey1:
#         print('OK')
#     else:
#         while k+1==numKey1:
#             strAnalysis=resDg.text[blockST[i]:blockED[i]]
#             res =re.search(patternRep, strAnalysis)
#             numKey1=int(res.group()[2:-2])
#             i=i+1
#         print('Corrected')
    if not(k==numKey0 and  k+1==numKey1):
        while k+1==numKey1:
            strAnalysis=resDg.text[blockST[i]:blockED[i]]
            res =re.search(patternRep, strAnalysis)
            numKey1=int(res.group()[2:-2])
            i=i+1
#         print('Corrected')
        
    strAnalysis=resDg.text[blockST[pre_i]:blockED[i]]
    k=k+1
    print([k+1, i+1])
    print(strAnalysis[1:10])
    print("---")

#     i=i+1


[2, 2]
1. J Med 
---
[3, 3]

2. Sci R
---
[4, 4]

3. Emerg
---
[5, 5]

4. Sci R
---
[6, 6]

5. J Med
---
[7, 7]

6. J Med
---
[8, 8]

7. Behav
---
[9, 9]

8. Lab A
---
[10, 10]

9. Neuro
---
[11, 11]

10. Fron
---
[12, 12]

11. Cell
---
[13, 13]

12. Mol 
---
[14, 14]

13. Neur
---
[15, 15]

14. Comm
---
[16, 16]

15. Gero
---
[17, 17]

16. NPJ 
---
[18, 18]

17. Sci 
---
[19, 19]

18. Fron
---
[20, 20]

19. Sci 
---
[21, 21]

20. Elif
---
[22, 22]

21. Infe
---
[23, 23]

22. J Co
---
[24, 24]

23. Tran
---
[25, 25]

24. J Am
---
[26, 26]

25. Acta
---
[27, 27]

26. J Ex
---
[28, 28]

27. Bioc
---
[29, 29]

28. ILAR
---
[30, 30]

29. J Vi
---
[31, 31]

30. PLoS
---
[32, 32]

31. Agin
---
[33, 33]

32. Nat 
---
[34, 34]

33. J Co
---
[35, 35]

34. Vet 
---
[36, 36]

35. Biom
---
[37, 37]

36. Ecoh
---
[38, 38]

37. Vacc
---
[39, 39]

38. Micr
---
[40, 40]

39. Nat 
---
[41, 41]

40. J Ve
---
[42, 42]

41. Phar
---
[43, 43]

42. Exp 
---
[44, 44]

43. Nat 
---
[45, 45]

44. Neur
---
[46,

[345, 345]

344. Rev
---
[346, 346]

345. Eur
---
[347, 347]

346. Ste
---
[348, 348]

347. Bio
---
[349, 349]

348. J N
---
[350, 350]

349. Sci
---
[351, 351]

350. Bio
---
[352, 352]

351. Nat
---
[353, 353]

352. Pri
---
[354, 354]

353. Pri
---
[355, 355]

354. Cer
---
[356, 356]

355. Ani
---
[357, 357]

356. BMC
---
[358, 358]

357. Hep
---
[359, 359]

358. Sci
---
[360, 360]

359. Par
---
[361, 361]

360. Exp
---
[362, 362]

361. PLo
---
[363, 363]

362. J P
---
[364, 364]

363. iSc
---
[365, 365]

364. Neu
---
[366, 366]

365. PLo
---
[367, 367]

366. Bio
---
[368, 368]

367. Mob
---
[369, 369]

368. Am 
---
[370, 370]

369. J M
---
[371, 371]

370. Bio
---
[372, 372]

371. Eur
---
[373, 373]

372. Dru
---
[374, 374]

373. Cry
---
[375, 375]

374. J N
---
[376, 376]

375. Tre
---
[377, 377]

376. Nat
---
[378, 378]

377. Eur
---
[379, 379]

378. Zoo
---
[380, 380]

379. Nat
---
[381, 381]

380. J M
---
[382, 382]

381. J M
---
[383, 383]

382. Imm
---
[384, 384]

383. Sci
---


[679, 681]

679. PLo
---
[680, 682]

680. Mic
---
[681, 683]

681. Nat
---
[682, 684]

682. Ant
---
[683, 685]

683. Ana
---
[684, 686]

684. Am 
---
[685, 687]

685. J P
---
[686, 688]

686. Mol
---
[687, 689]

687. Cel
---
[688, 690]

688. Hea
---
[689, 691]

689. Nat
---
[690, 692]

690. PLo
---
[691, 693]

691. eNe
---
[692, 694]

692. Phy
---
[693, 695]

693. Sci
---
[694, 696]

694. Par
---
[695, 697]

695. Psy
---
[696, 698]

696. Bra
---
[697, 699]

697. Mol
---
[698, 700]

698. Sci
---
[699, 701]

699. Vet
---
[700, 702]

700. Bio
---
[701, 703]

701. Ana
---
[702, 704]

702. Int
---
[703, 705]

703. Fro
---
[704, 706]

704. PLo
---
[705, 707]

705. Phy
---
[706, 708]

706. Bio
---
[707, 709]

707. Vir
---
[708, 710]

708. Pro
---
[709, 711]

709. J M
---
[710, 712]

710. PLo
---
[711, 713]

711. Sci
---
[712, 714]

712. Sci
---
[713, 715]

713. J N
---
[714, 716]

714. J V
---
[715, 717]

715. Am 
---
[716, 718]

716. J P
---
[717, 719]

717. J N
---
[718, 720]

718. Am 
---


[1006, 1012]

1006. PL
---
[1007, 1013]

1007. Sc
---
[1008, 1014]

1008. J 
---
[1009, 1015]

1009. Ti
---
[1010, 1016]

1010. De
---
[1011, 1017]

1011. De
---
[1012, 1018]

1012. Ne
---
[1013, 1019]

1013. Sc
---
[1014, 1020]

1014. Pa
---
[1015, 1021]

1015. Im
---
[1016, 1022]

1016. He
---
[1017, 1023]

1017. Xe
---
[1018, 1024]

1018. Ep
---
[1019, 1025]

1019. J 
---
[1020, 1026]

1020. J 
---
[1021, 1027]

1021. Ce
---
[1022, 1028]

1022. Bi
---
[1023, 1029]

1023. J 
---
[1024, 1030]

1024. Sc
---
[1025, 1031]

1025. Re
---
[1026, 1032]

1026. J 
---
[1027, 1033]

1027. Ce
---
[1028, 1034]

1028. Wo
---
[1029, 1035]

1029. Pa
---
[1030, 1036]

1030. Ne
---
[1031, 1037]

1031. Na
---
[1032, 1038]

1032. RN
---
[1033, 1039]

1033. J 
---
[1034, 1040]

1034. La
---
[1035, 1041]

1035. Bi
---
[1036, 1042]

1036. Bi
---
[1037, 1043]

1037. Ar
---
[1038, 1044]

1038. Co
---
[1039, 1045]

1039. J 
---
[1040, 1046]

1040. Am
---
[1041, 1047]

1041. J 
---
[1042, 1048]

1042. Re
---
[

[1310, 1316]

1310. Ge
---
[1311, 1317]

1311. Am
---
[1312, 1318]

1312. Am
---
[1313, 1319]

1313. J 
---
[1314, 1320]

1314. Na
---
[1315, 1321]

1315. Eu
---
[1316, 1322]

1316. Pa
---
[1317, 1323]

1317. Ne
---
[1318, 1324]

1318. In
---
[1319, 1325]

1319. Na
---
[1320, 1326]

1320. Br
---
[1321, 1327]

1321. Cu
---
[1322, 1328]

1322. Ne
---
[1323, 1329]

1323. Ne
---
[1324, 1330]

1324. Ph
---
[1325, 1331]

1325. Ne
---
[1326, 1332]

1326. Ph
---
[1327, 1333]

1327. Ex
---
[1328, 1334]

1328. On
---
[1329, 1335]

1329. Ne
---
[1330, 1336]

1330. Sc
---
[1331, 1337]

1331. Re
---
[1332, 1338]

1332. Am
---
[1333, 1339]

1333. Co
---
[1334, 1340]

1334. Cl
---
[1335, 1341]

1335. J 
---
[1336, 1342]

1336. J 
---
[1337, 1343]

1337. PL
---
[1338, 1344]

1338. Ti
---
[1339, 1345]

1339. Na
---
[1340, 1346]

1340. An
---
[1341, 1347]

1341. Up
---
[1342, 1348]

1342. Mi
---
[1343, 1349]

1343. Ne
---
[1344, 1350]

1344. An
---
[1345, 1351]

1345. Ne
---
[1346, 1352]

1346. Ne
---
[

...

[4074, 4118]

4074. J 
---
[4075, 4119]

4075. Ne
---
[4076, 4120]

4076. La
---
[4077, 4121]

4077. In
---
[4078, 4122]

4078. An
---
[4079, 4123]

4079. An
---
[4080, 4124]

4080. Bi
---
[4081, 4125]

4081. Eu
---
[4082, 4126]

4082. J 
---
[4083, 4127]

4083. J 
---
[4084, 4128]

4084. J 
---
[4085, 4129]

4085. Ph
---
[4086, 4130]

4086. J 
---
[4087, 4131]

4087. Pr
---
[4088, 4132]

4088. Mo
---
[4089, 4133]

4089. Bi
---
[4090, 4134]

4090. Ne
---
[4091, 4135]

4091. Vi
---
[4092, 4136]

4092. Vi
---
[4093, 4137]

4093. Ph
---
[4094, 4138]

4094. Am
---
[4095, 4139]

4095. Ne
---
[4096, 4140]

4096. J 
---
[4097, 4141]

4097. Ne
---
[4098, 4142]

4098. Eu
---
[4099, 4143]

4099. Bi
---
[4100, 4144]

4100. J 
---
[4101, 4145]

4101. J 
---
[4102, 4146]

4102. Vi
---
[4103, 4147]

4103. Vi
---
[4104, 4148]

4104. J 
---
[4105, 4149]

4105. An
---
[4106, 4150]

4106. Ci
---
[4107, 4151]

4107. Hy
---
[4108, 4152]

4108. J 
---
[4109, 4153]

4109. Ho
---
[4110, 4154]

4110. Ph
---
[

[4379, 4429]

4378. Ar
---
[4380, 4430]

4379. An
---
[4381, 4431]

4380. Sc
---
[4382, 4432]

4381. Fo
---
[4383, 4433]

4382. Ar
---
[4384, 4434]

4383. Ar
---
[4385, 4435]

4384. Co
---
[4386, 4438]

1. Hydro
---
[4387, 4439]

4387. Ar
---
[4388, 4440]

4388. Fo
---
[4389, 4441]

4389. La
---
[4390, 4442]

4390. Fo
---
[4391, 4443]

4391. Ad
---
[4392, 4444]

4392. Ch
---
[4393, 4445]

4393. Ex
---
[4394, 4446]

4394. Bi
---
[4395, 4447]

4395. Ne
---
[4396, 4448]

4396. Ri
---
[4397, 4449]

4397. La
---
[4398, 4450]

4398. J 
---
[4399, 4451]

4399. Hi
---
[4400, 4452]

4400. J 
---
[4401, 4453]

4401. Ex
---
[4402, 4454]

4402. Eu
---
[4403, 4455]

4403. J 
---
[4404, 4456]

4404. J 
---
[4405, 4457]

4405. Ac
---
[4406, 4458]

4406. Bi
---
[4407, 4459]

4407. Bi
---
[4408, 4460]

4408. Co
---
[4409, 4461]

4409. Co
---
[4410, 4462]

4410. J 
---
[4411, 4463]

4411. Am
---
[4412, 4464]

4412. Bi
---
[4413, 4465]

4413. Am
---
[4414, 4466]

4414. Z 
---
[4415, 4467]

4415. Ho
---
[

[4685, 4737]

4684. Am
---
[4686, 4738]

4685. Am
---
[4687, 4739]

4686. Am
---
[4688, 4740]

4687. Am
---
[4689, 4741]

4688. Am
---
[4690, 4742]

4689. Ad
---
[4691, 4743]

4690. Pr
---
[4692, 4744]

4691. Fo
---
[4693, 4745]

4692. Ps
---
[4694, 4746]

4693. Fo
---
[4695, 4747]

4694. Ne
---
[4696, 4748]

4695. J 
---
[4697, 4749]

4696. Im
---
[4698, 4750]

4697. J 
---
[4699, 4751]

4698. Cl
---
[4700, 4752]

4699. An
---
[4701, 4753]

4700. Co
---
[4702, 4754]

4701. Co
---
[4703, 4755]

4702. An
---
[4704, 4756]

4703. J 
---
[4705, 4757]

4704. J 
---
[4706, 4758]

4705. Bi
---
[4707, 4759]

4706. J 
---
[4708, 4760]

4707. J 
---
[4709, 4761]

4708. Re
---
[4710, 4762]

4709. Be
---
[4711, 4763]

4710. La
---
[4712, 4764]

4711. La
---
[4713, 4765]

4712. La
---
[4714, 4766]

4713. In
---
[4715, 4767]

4714. Ar
---
[4716, 4768]

4715. J 
---
[4717, 4769]

4716. J 
---
[4718, 4770]

4717. Ps
---
[4719, 4771]

4718. J 
---
[4720, 4772]

4719. La
---
[4721, 4773]

4720. Ph
---
[

[4992, 5050]

4991. La
---
[4993, 5051]

4992. J 
---
[4994, 5052]

4993. Am
---
[4995, 5053]

4994. J 
---
[4996, 5054]

4995. Bi
---
[4997, 5055]

4996. Bi
---
[4998, 5056]

4997. La
---
[4999, 5057]

4998. Cy
---
[5000, 5058]

4999. Ac
---
[5001, 5059]

5000. Sc
---
[5002, 5060]

5001. Re
---
[5003, 5061]

5002. Cy
---
[5004, 5062]

5003. En
---
[5005, 5063]

5004. Pr
---
[5006, 5064]

5005. Bi
---
[5007, 5065]

5006. Sc
---
[5008, 5066]

5007. Am
---
[5009, 5067]

5008. Bi
---
[5010, 5068]

5009. J 
---
[5011, 5069]

5010. J 
---
[5012, 5070]

5011. Am
---
[5013, 5071]

5012. J 
---
[5014, 5072]

5013. J 
---
[5015, 5073]

5014. J 
---
[5016, 5074]

5015. Am
---
[5017, 5075]

5016. Or
---
[5018, 5076]

5017. AM
---
[5019, 5077]

5018. Ho
---
[5020, 5078]

5019. Zh
---
[5021, 5079]

5020. J 
---
[5022, 5080]

5021. J 
---
[5023, 5081]

5022. Am
---
[5024, 5082]

5023. J 
---


IndexError: list index out of range

In [144]:
i=2609-1
k=2595-1
# for i in range(len(blockST)):
# while i<len(blockST):
while i<2615:
    fout_name=os.path.join(fname_tag,str(k+1)+fext)
    patternRep="\n\n(\d+). "
    if i==0:
        patternRep="\n(\d+). "
    else:
        patternRep="\n\n(\d+). "  
#     strAnalysis=resDg.text[blockST[i]:blockED[i]]
#     res =re.search(patternRep, strAnalysis)
#     numKey=int(res.group()[2:-2])
#     print(numKey)
    
#     k=581
#     i=581
#     strAnalysis=resDg.text[blockST[i]:blockED[i]]
#     print(strAnalysis)
#     patternRep="\n\n(\d+). "
    strAnalysis=resDg.text[blockST[i]:blockED[i]]
    res =re.search(patternRep, strAnalysis)
    if i==0:
        hpos=1
    else:
        hpos=2 
    numKey0=int(res.group()[hpos:-2])
#     print(numKey0)
    pre_i=i
    i=i+1
    patternRep="\n\n(\d+). "
    strAnalysis=resDg.text[blockST[i]:blockED[i]]
    res =re.search(patternRep, strAnalysis)
    numKey1=int(res.group()[2:-2])

#     if k==numKey0 and  k+1==numKey1:
#         print('OK')
#     else:
#         while k+1==numKey1:
#             strAnalysis=resDg.text[blockST[i]:blockED[i]]
#             res =re.search(patternRep, strAnalysis)
#             numKey1=int(res.group()[2:-2])
#             i=i+1
#         print('Corrected')
#     if (k+1)!=numKey1:
    while (k+2)!=numKey1:
        i=i+1
        strAnalysis=resDg.text[blockST[i]:blockED[i]]
        res =re.search(patternRep, strAnalysis)
        numKey1=int(res.group()[2:-2])

#         print('Corrected')
        
    strAnalysis=resDg.text[blockST[pre_i]:blockED[i]]
    k=k+1
    print([k,i])
    print(strAnalysis[1:10])
    print("---")

#     i=i+1


[2595, 2609]

2595. Am
---
[2596, 2611]

2596. NM
---
[2597, 2612]

2597. Mo
---
[2598, 2613]

2598. Ac
---
[2599, 2614]

2599. Br
---
[2600, 2615]

2600. Bi
---


In [139]:
print(numKey0)
print(numKey1)
print(k+2)
print((k+2)!=numKey1)
print('---')

i=i+1
strAnalysis=resDg.text[blockST[i]:blockED[i]]
res =re.search(patternRep, strAnalysis)
numKey1=int(res.group()[2:-2])
print(numKey1)
print(k+1)
print((k+1)!=numKey1)
print('---')


2595
2596
2596
False
---
200
2595
True
---


In [109]:
print(numKey0)
print(numKey1)
print(k)

200
2597
2594


In [145]:
i=0
k=0
# for i in range(len(blockST)):
while i<resNum:
# while i<2:
    fout_name=os.path.join(fname_tag,str(k+1)+fext)
    patternRep="\n\n(\d+). "
    if i==0:
        patternRep="\n(\d+). "
    else:
        patternRep="\n\n(\d+). "  
#     strAnalysis=resDg.text[blockST[i]:blockED[i]]
#     res =re.search(patternRep, strAnalysis)
#     numKey=int(res.group()[2:-2])
#     print(numKey)
    
#     k=581
#     i=581
#     strAnalysis=resDg.text[blockST[i]:blockED[i]]
#     print(strAnalysis)
#     patternRep="\n\n(\d+). "
    strAnalysis=resDg.text[blockST[i]:blockED[i]]
    res =re.search(patternRep, strAnalysis)
    if i==0:
        hpos=1
    else:
        hpos=2 
    numKey0=int(res.group()[hpos:-2])
#     print(numKey0)
    pre_i=i
    i=i+1
    patternRep="\n\n(\d+). "
    strAnalysis=resDg.text[blockST[i]:blockED[i]]
    res =re.search(patternRep, strAnalysis)
    numKey1=int(res.group()[2:-2])

#     if k==numKey0 and  k+1==numKey1:
#         print('OK')
#     else:
#         while k+1==numKey1:
#             strAnalysis=resDg.text[blockST[i]:blockED[i]]
#             res =re.search(patternRep, strAnalysis)
#             numKey1=int(res.group()[2:-2])
#             i=i+1
#         print('Corrected')
    while (k+2)!=numKey1:
        i=i+1
        strAnalysis=resDg.text[blockST[i]:blockED[i]]
        res =re.search(patternRep, strAnalysis)
        numKey1=int(res.group()[2:-2])
        
#         print('Corrected')
        
    strAnalysis=resDg.text[blockST[pre_i]:blockED[i]]
    k=k+1
    print([k, i])
    print(strAnalysis[1:10])
    print("---")

#     i=i+1


[1, 1]
1. J Med 
---
[2, 2]

2. Sci R
---
[3, 3]

3. Emerg
---
[4, 4]

4. Sci R
---
[5, 5]

5. J Med
---
[6, 6]

6. J Med
---
[7, 7]

7. Behav
---
[8, 8]

8. Lab A
---
[9, 9]

9. Neuro
---
[10, 10]

10. Fron
---
[11, 11]

11. Cell
---
[12, 12]

12. Mol 
---
[13, 13]

13. Neur
---
[14, 14]

14. Comm
---
[15, 15]

15. Gero
---
[16, 16]

16. NPJ 
---
[17, 17]

17. Sci 
---
[18, 18]

18. Fron
---
[19, 19]

19. Sci 
---
[20, 20]

20. Elif
---
[21, 21]

21. Infe
---
[22, 22]

22. J Co
---
[23, 23]

23. Tran
---
[24, 24]

24. J Am
---
[25, 25]

25. Acta
---
[26, 26]

26. J Ex
---
[27, 27]

27. Bioc
---
[28, 28]

28. ILAR
---
[29, 29]

29. J Vi
---
[30, 30]

30. PLoS
---
[31, 31]

31. Agin
---
[32, 32]

32. Nat 
---
[33, 33]

33. J Co
---
[34, 34]

34. Vet 
---
[35, 35]

35. Biom
---
[36, 36]

36. Ecoh
---
[37, 37]

37. Vacc
---
[38, 38]

38. Micr
---
[39, 39]

39. Nat 
---
[40, 40]

40. J Ve
---
[41, 41]

41. Phar
---
[42, 42]

42. Exp 
---
[43, 43]

43. Nat 
---
[44, 44]

44. Neur
---
[45, 4

[338, 338]

338. Pat
---
[339, 339]

339. Fro
---
[340, 340]

340. mSy
---
[341, 341]

341. Ana
---
[342, 342]

342. Sci
---
[343, 343]

343. Ste
---
[344, 344]

344. Rev
---
[345, 345]

345. Eur
---
[346, 346]

346. Ste
---
[347, 347]

347. Bio
---
[348, 348]

348. J N
---
[349, 349]

349. Sci
---
[350, 350]

350. Bio
---
[351, 351]

351. Nat
---
[352, 352]

352. Pri
---
[353, 353]

353. Pri
---
[354, 354]

354. Cer
---
[355, 355]

355. Ani
---
[356, 356]

356. BMC
---
[357, 357]

357. Hep
---
[358, 358]

358. Sci
---
[359, 359]

359. Par
---
[360, 360]

360. Exp
---
[361, 361]

361. PLo
---
[362, 362]

362. J P
---
[363, 363]

363. iSc
---
[364, 364]

364. Neu
---
[365, 365]

365. PLo
---
[366, 366]

366. Bio
---
[367, 367]

367. Mob
---
[368, 368]

368. Am 
---
[369, 369]

369. J M
---
[370, 370]

370. Bio
---
[371, 371]

371. Eur
---
[372, 372]

372. Dru
---
[373, 373]

373. Cry
---
[374, 374]

374. J N
---
[375, 375]

375. Tre
---
[376, 376]

376. Nat
---
[377, 377]

377. Eur
---


[670, 671]

670. IEE
---
[671, 672]

671. J V
---
[672, 673]

672. Sci
---
[673, 674]

673. J C
---
[674, 675]

674. ACS
---
[675, 676]

675. Neu
---
[676, 677]

676. Nat
---
[677, 678]

677. Ani
---
[678, 679]

678. PLo
---
[679, 680]

679. PLo
---
[680, 681]

680. Mic
---
[681, 682]

681. Nat
---
[682, 683]

682. Ant
---
[683, 684]

683. Ana
---
[684, 685]

684. Am 
---
[685, 686]

685. J P
---
[686, 687]

686. Mol
---
[687, 688]

687. Cel
---
[688, 689]

688. Hea
---
[689, 690]

689. Nat
---
[690, 691]

690. PLo
---
[691, 692]

691. eNe
---
[692, 693]

692. Phy
---
[693, 694]

693. Sci
---
[694, 695]

694. Par
---
[695, 696]

695. Psy
---
[696, 697]

696. Bra
---
[697, 698]

697. Mol
---
[698, 699]

698. Sci
---
[699, 700]

699. Vet
---
[700, 701]

700. Bio
---
[701, 702]

701. Ana
---
[702, 703]

702. Int
---
[703, 704]

703. Fro
---
[704, 705]

704. PLo
---
[705, 706]

705. Phy
---
[706, 707]

706. Bio
---
[707, 708]

707. Vir
---
[708, 709]

708. Pro
---
[709, 710]

709. J M
---


[1000, 1005]

1000. Fr
---
[1001, 1006]

1001. Sc
---
[1002, 1007]

1002. Cu
---
[1003, 1008]

1003. Am
---
[1004, 1009]

1004. Pr
---
[1005, 1010]

1005. Ma
---
[1006, 1011]

1006. PL
---
[1007, 1012]

1007. Sc
---
[1008, 1013]

1008. J 
---
[1009, 1014]

1009. Ti
---
[1010, 1015]

1010. De
---
[1011, 1016]

1011. De
---
[1012, 1017]

1012. Ne
---
[1013, 1018]

1013. Sc
---
[1014, 1019]

1014. Pa
---
[1015, 1020]

1015. Im
---
[1016, 1021]

1016. He
---
[1017, 1022]

1017. Xe
---
[1018, 1023]

1018. Ep
---
[1019, 1024]

1019. J 
---
[1020, 1025]

1020. J 
---
[1021, 1026]

1021. Ce
---
[1022, 1027]

1022. Bi
---
[1023, 1028]

1023. J 
---
[1024, 1029]

1024. Sc
---
[1025, 1030]

1025. Re
---
[1026, 1031]

1026. J 
---
[1027, 1032]

1027. Ce
---
[1028, 1033]

1028. Wo
---
[1029, 1034]

1029. Pa
---
[1030, 1035]

1030. Ne
---
[1031, 1036]

1031. Na
---
[1032, 1037]

1032. RN
---
[1033, 1038]

1033. J 
---
[1034, 1039]

1034. La
---
[1035, 1040]

1035. Bi
---
[1036, 1041]

1036. Bi
---
[

[1306, 1311]

1306. J 
---
[1307, 1312]

1307. Pr
---
[1308, 1313]

1308. Me
---
[1309, 1314]

1309. Em
---
[1310, 1315]

1310. Ge
---
[1311, 1316]

1311. Am
---
[1312, 1317]

1312. Am
---
[1313, 1318]

1313. J 
---
[1314, 1319]

1314. Na
---
[1315, 1320]

1315. Eu
---
[1316, 1321]

1316. Pa
---
[1317, 1322]

1317. Ne
---
[1318, 1323]

1318. In
---
[1319, 1324]

1319. Na
---
[1320, 1325]

1320. Br
---
[1321, 1326]

1321. Cu
---
[1322, 1327]

1322. Ne
---
[1323, 1328]

1323. Ne
---
[1324, 1329]

1324. Ph
---
[1325, 1330]

1325. Ne
---
[1326, 1331]

1326. Ph
---
[1327, 1332]

1327. Ex
---
[1328, 1333]

1328. On
---
[1329, 1334]

1329. Ne
---
[1330, 1335]

1330. Sc
---
[1331, 1336]

1331. Re
---
[1332, 1337]

1332. Am
---
[1333, 1338]

1333. Co
---
[1334, 1339]

1334. Cl
---
[1335, 1340]

1335. J 
---
[1336, 1341]

1336. J 
---
[1337, 1342]

1337. PL
---
[1338, 1343]

1338. Ti
---
[1339, 1344]

1339. Na
---
[1340, 1345]

1340. An
---
[1341, 1346]

1341. Up
---
[1342, 1347]

1342. Mi
---
[

...

[4074, 4117]

4074. J 
---
[4075, 4118]

4075. Ne
---
[4076, 4119]

4076. La
---
[4077, 4120]

4077. In
---
[4078, 4121]

4078. An
---
[4079, 4122]

4079. An
---
[4080, 4123]

4080. Bi
---
[4081, 4124]

4081. Eu
---
[4082, 4125]

4082. J 
---
[4083, 4126]

4083. J 
---
[4084, 4127]

4084. J 
---
[4085, 4128]

4085. Ph
---
[4086, 4129]

4086. J 
---
[4087, 4130]

4087. Pr
---
[4088, 4131]

4088. Mo
---
[4089, 4132]

4089. Bi
---
[4090, 4133]

4090. Ne
---
[4091, 4134]

4091. Vi
---
[4092, 4135]

4092. Vi
---
[4093, 4136]

4093. Ph
---
[4094, 4137]

4094. Am
---
[4095, 4138]

4095. Ne
---
[4096, 4139]

4096. J 
---
[4097, 4140]

4097. Ne
---
[4098, 4141]

4098. Eu
---
[4099, 4142]

4099. Bi
---
[4100, 4143]

4100. J 
---
[4101, 4144]

4101. J 
---
[4102, 4145]

4102. Vi
---
[4103, 4146]

4103. Vi
---
[4104, 4147]

4104. J 
---
[4105, 4148]

4105. An
---
[4106, 4149]

4106. Ci
---
[4107, 4150]

4107. Hy
---
[4108, 4151]

4108. J 
---
[4109, 4152]

4109. Ho
---
[4110, 4153]

4110. Ph
---
[

[4379, 4429]

4379. An
---
[4380, 4430]

4380. Sc
---
[4381, 4431]

4381. Fo
---
[4382, 4432]

4382. Ar
---
[4383, 4433]

4383. Ar
---
[4384, 4435]

4384. Co
---
[4385, 4436]

4385. Ac
---
[4386, 4437]

4386. J 
---
[4387, 4438]

4387. Ar
---
[4388, 4439]

4388. Fo
---
[4389, 4440]

4389. La
---
[4390, 4441]

4390. Fo
---
[4391, 4442]

4391. Ad
---
[4392, 4443]

4392. Ch
---
[4393, 4444]

4393. Ex
---
[4394, 4445]

4394. Bi
---
[4395, 4446]

4395. Ne
---
[4396, 4447]

4396. Ri
---
[4397, 4448]

4397. La
---
[4398, 4449]

4398. J 
---
[4399, 4450]

4399. Hi
---
[4400, 4451]

4400. J 
---
[4401, 4452]

4401. Ex
---
[4402, 4453]

4402. Eu
---
[4403, 4454]

4403. J 
---
[4404, 4455]

4404. J 
---
[4405, 4456]

4405. Ac
---
[4406, 4457]

4406. Bi
---
[4407, 4458]

4407. Bi
---
[4408, 4459]

4408. Co
---
[4409, 4460]

4409. Co
---
[4410, 4461]

4410. J 
---
[4411, 4462]

4411. Am
---
[4412, 4463]

4412. Bi
---
[4413, 4464]

4413. Am
---
[4414, 4465]

4414. Z 
---
[4415, 4466]

4415. Ho
---
[

[4688, 4740]

4688. Am
---
[4689, 4741]

4689. Ad
---
[4690, 4742]

4690. Pr
---
[4691, 4743]

4691. Fo
---
[4692, 4744]

4692. Ps
---
[4693, 4745]

4693. Fo
---
[4694, 4746]

4694. Ne
---
[4695, 4747]

4695. J 
---
[4696, 4748]

4696. Im
---
[4697, 4749]

4697. J 
---
[4698, 4750]

4698. Cl
---
[4699, 4751]

4699. An
---
[4700, 4752]

4700. Co
---
[4701, 4753]

4701. Co
---
[4702, 4754]

4702. An
---
[4703, 4755]

4703. J 
---
[4704, 4756]

4704. J 
---
[4705, 4757]

4705. Bi
---
[4706, 4758]

4706. J 
---
[4707, 4759]

4707. J 
---
[4708, 4760]

4708. Re
---
[4709, 4761]

4709. Be
---
[4710, 4762]

4710. La
---
[4711, 4763]

4711. La
---
[4712, 4764]

4712. La
---
[4713, 4765]

4713. In
---
[4714, 4766]

4714. Ar
---
[4715, 4767]

4715. J 
---
[4716, 4768]

4716. J 
---
[4717, 4769]

4717. Ps
---
[4718, 4770]

4718. J 
---
[4719, 4771]

4719. La
---
[4720, 4772]

4720. Ph
---
[4721, 4773]

4721. Ph
---
[4722, 4774]

4722. Re
---
[4723, 4775]

4723. Ac
---
[4724, 4776]

4724. La
---
[

[4998, 5056]

4998. Cy
---
[4999, 5057]

4999. Ac
---
[5000, 5058]

5000. Sc
---
[5001, 5059]

5001. Re
---
[5002, 5060]

5002. Cy
---
[5003, 5061]

5003. En
---
[5004, 5062]

5004. Pr
---
[5005, 5063]

5005. Bi
---
[5006, 5064]

5006. Sc
---
[5007, 5065]

5007. Am
---
[5008, 5066]

5008. Bi
---
[5009, 5067]

5009. J 
---
[5010, 5068]

5010. J 
---
[5011, 5069]

5011. Am
---
[5012, 5070]

5012. J 
---
[5013, 5071]

5013. J 
---
[5014, 5072]

5014. J 
---
[5015, 5073]

5015. Am
---
[5016, 5074]

5016. Or
---
[5017, 5075]

5017. AM
---
[5018, 5076]

5018. Ho
---
[5019, 5077]

5019. Zh
---
[5020, 5078]

5020. J 
---
[5021, 5079]

5021. J 
---
[5022, 5080]

5022. Am
---


IndexError: list index out of range

In [147]:
resNum

5023

In [177]:
i=0
k=0
while k<resNum:
    fout_name=os.path.join(fname_tag,str(k+1)+fext)
    patternRep="\n\n(\d+). "
    if i==0:
        patternRep="\n(\d+). "
    else:
        patternRep="\n\n(\d+). "  

    strAnalysis=resDg.text[blockST[i]:blockED[i]]
    res =re.search(patternRep, strAnalysis)
    if i==0:
        hpos=1
    else:
        hpos=2 
    numKey0=int(res.group()[hpos:-2])
#     print(numKey0)
    pre_i=i
    i=i+1
    patternRep="\n\n(\d+). "
    strAnalysis=resDg.text[blockST[i]:blockED[i]]
    res =re.search(patternRep, strAnalysis)
    numKey1=int(res.group()[2:-2])

    while (k+2)!=numKey1 and k!=resNum-1:
        i=i+1
        strAnalysis=resDg.text[blockST[i]:blockED[i]]
        res =re.search(patternRep, strAnalysis)
        numKey1=int(res.group()[2:-2])
    if i>len(blockED)-1:
        i=len(blockED)-1
    strAnalysis=resDg.text[blockST[pre_i]:blockED[i]]
    fout_name=os.path.join(fname_tag,str(k+1)+fext)
    with open(fout_name, "w", encoding="utf-8") as fid:
        fid.write(strAnalysis)
    
    k=k+1
    print([k, i])
    print(strAnalysis[1:10])
    print("---")



[1, 1]
1. J Med 
---
[2, 2]

2. Sci R
---
[3, 3]

3. Emerg
---
[4, 4]

4. Sci R
---
[5, 5]

5. J Med
---
[6, 6]

6. J Med
---
[7, 7]

7. Behav
---
[8, 8]

8. Lab A
---
[9, 9]

9. Neuro
---
[10, 10]

10. Fron
---
[11, 11]

11. Cell
---
[12, 12]

12. Mol 
---
[13, 13]

13. Neur
---
[14, 14]

14. Comm
---
[15, 15]

15. Gero
---
[16, 16]

16. NPJ 
---
[17, 17]

17. Sci 
---
[18, 18]

18. Fron
---
[19, 19]

19. Sci 
---
[20, 20]

20. Elif
---
[21, 21]

21. Infe
---
[22, 22]

22. J Co
---
[23, 23]

23. Tran
---
[24, 24]

24. J Am
---
[25, 25]

25. Acta
---
[26, 26]

26. J Ex
---
[27, 27]

27. Bioc
---
[28, 28]

28. ILAR
---
[29, 29]

29. J Vi
---
[30, 30]

30. PLoS
---
[31, 31]

31. Agin
---
[32, 32]

32. Nat 
---
[33, 33]

33. J Co
---
[34, 34]

34. Vet 
---
[35, 35]

35. Biom
---
[36, 36]

36. Ecoh
---
[37, 37]

37. Vacc
---
[38, 38]

38. Micr
---
[39, 39]

39. Nat 
---
[40, 40]

40. J Ve
---
[41, 41]

41. Phar
---
[42, 42]

42. Exp 
---
[43, 43]

43. Nat 
---
[44, 44]

44. Neur
---
[45, 4

[343, 343]

343. Ste
---
[344, 344]

344. Rev
---
[345, 345]

345. Eur
---
[346, 346]

346. Ste
---
[347, 347]

347. Bio
---
[348, 348]

348. J N
---
[349, 349]

349. Sci
---
[350, 350]

350. Bio
---
[351, 351]

351. Nat
---
[352, 352]

352. Pri
---
[353, 353]

353. Pri
---
[354, 354]

354. Cer
---
[355, 355]

355. Ani
---
[356, 356]

356. BMC
---
[357, 357]

357. Hep
---
[358, 358]

358. Sci
---
[359, 359]

359. Par
---
[360, 360]

360. Exp
---
[361, 361]

361. PLo
---
[362, 362]

362. J P
---
[363, 363]

363. iSc
---
[364, 364]

364. Neu
---
[365, 365]

365. PLo
---
[366, 366]

366. Bio
---
[367, 367]

367. Mob
---
[368, 368]

368. Am 
---
[369, 369]

369. J M
---
[370, 370]

370. Bio
---
[371, 371]

371. Eur
---
[372, 372]

372. Dru
---
[373, 373]

373. Cry
---
[374, 374]

374. J N
---
[375, 375]

375. Tre
---
[376, 376]

376. Nat
---
[377, 377]

377. Eur
---
[378, 378]

378. Zoo
---
[379, 379]

379. Nat
---
[380, 380]

380. J M
---
[381, 381]

381. J M
---
[382, 382]

382. Imm
---


[674, 675]

674. ACS
---
[675, 676]

675. Neu
---
[676, 677]

676. Nat
---
[677, 678]

677. Ani
---
[678, 679]

678. PLo
---
[679, 680]

679. PLo
---
[680, 681]

680. Mic
---
[681, 682]

681. Nat
---
[682, 683]

682. Ant
---
[683, 684]

683. Ana
---
[684, 685]

684. Am 
---
[685, 686]

685. J P
---
[686, 687]

686. Mol
---
[687, 688]

687. Cel
---
[688, 689]

688. Hea
---
[689, 690]

689. Nat
---
[690, 691]

690. PLo
---
[691, 692]

691. eNe
---
[692, 693]

692. Phy
---
[693, 694]

693. Sci
---
[694, 695]

694. Par
---
[695, 696]

695. Psy
---
[696, 697]

696. Bra
---
[697, 698]

697. Mol
---
[698, 699]

698. Sci
---
[699, 700]

699. Vet
---
[700, 701]

700. Bio
---
[701, 702]

701. Ana
---
[702, 703]

702. Int
---
[703, 704]

703. Fro
---
[704, 705]

704. PLo
---
[705, 706]

705. Phy
---
[706, 707]

706. Bio
---
[707, 708]

707. Vir
---
[708, 709]

708. Pro
---
[709, 710]

709. J M
---
[710, 711]

710. PLo
---
[711, 712]

711. Sci
---
[712, 713]

712. Sci
---
[713, 714]

713. J N
---


[1007, 1012]

1007. Sc
---
[1008, 1013]

1008. J 
---
[1009, 1014]

1009. Ti
---
[1010, 1015]

1010. De
---
[1011, 1016]

1011. De
---
[1012, 1017]

1012. Ne
---
[1013, 1018]

1013. Sc
---
[1014, 1019]

1014. Pa
---
[1015, 1020]

1015. Im
---
[1016, 1021]

1016. He
---
[1017, 1022]

1017. Xe
---
[1018, 1023]

1018. Ep
---
[1019, 1024]

1019. J 
---
[1020, 1025]

1020. J 
---
[1021, 1026]

1021. Ce
---
[1022, 1027]

1022. Bi
---
[1023, 1028]

1023. J 
---
[1024, 1029]

1024. Sc
---
[1025, 1030]

1025. Re
---
[1026, 1031]

1026. J 
---
[1027, 1032]

1027. Ce
---
[1028, 1033]

1028. Wo
---
[1029, 1034]

1029. Pa
---
[1030, 1035]

1030. Ne
---
[1031, 1036]

1031. Na
---
[1032, 1037]

1032. RN
---
[1033, 1038]

1033. J 
---
[1034, 1039]

1034. La
---
[1035, 1040]

1035. Bi
---
[1036, 1041]

1036. Bi
---
[1037, 1042]

1037. Ar
---
[1038, 1043]

1038. Co
---
[1039, 1044]

1039. J 
---
[1040, 1045]

1040. Am
---
[1041, 1046]

1041. J 
---
[1042, 1047]

1042. Re
---
[1043, 1048]

1043. Bi
---
[

[1315, 1320]

1315. Eu
---
[1316, 1321]

1316. Pa
---
[1317, 1322]

1317. Ne
---
[1318, 1323]

1318. In
---
[1319, 1324]

1319. Na
---
[1320, 1325]

1320. Br
---
[1321, 1326]

1321. Cu
---
[1322, 1327]

1322. Ne
---
[1323, 1328]

1323. Ne
---
[1324, 1329]

1324. Ph
---
[1325, 1330]

1325. Ne
---
[1326, 1331]

1326. Ph
---
[1327, 1332]

1327. Ex
---
[1328, 1333]

1328. On
---
[1329, 1334]

1329. Ne
---
[1330, 1335]

1330. Sc
---
[1331, 1336]

1331. Re
---
[1332, 1337]

1332. Am
---
[1333, 1338]

1333. Co
---
[1334, 1339]

1334. Cl
---
[1335, 1340]

1335. J 
---
[1336, 1341]

1336. J 
---
[1337, 1342]

1337. PL
---
[1338, 1343]

1338. Ti
---
[1339, 1344]

1339. Na
---
[1340, 1345]

1340. An
---
[1341, 1346]

1341. Up
---
[1342, 1347]

1342. Mi
---
[1343, 1348]

1343. Ne
---
[1344, 1349]

1344. An
---
[1345, 1350]

1345. Ne
---
[1346, 1351]

1346. Ne
---
[1347, 1352]

1347. Na
---
[1348, 1353]

1348. Br
---
[1349, 1354]

1349. De
---
[1350, 1355]

1350. Na
---
[1351, 1356]

1351. Ne
---
[

...

[3770, 3805]

3770. In
---
[3771, 3806]

3771. Bi
---
[3772, 3807]

3772. Ti
---
[3773, 3808]

3773. La
---
[3774, 3809]

3774. Me
---
[3775, 3810]

3775. La
---
[3776, 3811]

3776. Sy
---
[3777, 3812]

3777. J 
---
[3778, 3813]

3778. Ex
---
[3779, 3814]

3779. Bi
---
[3780, 3815]

3780. Na
---
[3781, 3816]

3781. Ac
---
[3782, 3817]

3782. La
---
[3783, 3818]

3783. La
---
[3784, 3819]

3784. Am
---
[3785, 3820]

3785. Pe
---
[3786, 3821]

3786. J 
---
[3787, 3822]

3787. J 
---
[3788, 3823]

3788. Tr
---
[3789, 3824]

3789. J 
---
[3790, 3825]

3790. Hy
---
[3791, 3826]

3791. Br
---
[3792, 3827]

3792. Co
---
[3793, 3828]

3793. Ge
---
[3794, 3829]

3794. J 
---
[3795, 3830]

3795. La
---
[3796, 3831]

3796. In
---
[3797, 3832]

3797. Bi
---
[3798, 3834]

3798. Br
---
[3799, 3835]

3799. La
---
[3800, 3836]

3800. An
---
[3801, 3837]

3801. J 
---
[3802, 3838]

3802. Na
---
[3803, 3839]

3803. Am
---
[3804, 3840]

3804. J 
---
[3805, 3841]

3805. J 
---
[3806, 3842]

3806. Vi
---
[

[4075, 4118]

4075. Ne
---
[4076, 4119]

4076. La
---
[4077, 4120]

4077. In
---
[4078, 4121]

4078. An
---
[4079, 4122]

4079. An
---
[4080, 4123]

4080. Bi
---
[4081, 4124]

4081. Eu
---
[4082, 4125]

4082. J 
---
[4083, 4126]

4083. J 
---
[4084, 4127]

4084. J 
---
[4085, 4128]

4085. Ph
---
[4086, 4129]

4086. J 
---
[4087, 4130]

4087. Pr
---
[4088, 4131]

4088. Mo
---
[4089, 4132]

4089. Bi
---
[4090, 4133]

4090. Ne
---
[4091, 4134]

4091. Vi
---
[4092, 4135]

4092. Vi
---
[4093, 4136]

4093. Ph
---
[4094, 4137]

4094. Am
---
[4095, 4138]

4095. Ne
---
[4096, 4139]

4096. J 
---
[4097, 4140]

4097. Ne
---
[4098, 4141]

4098. Eu
---
[4099, 4142]

4099. Bi
---
[4100, 4143]

4100. J 
---
[4101, 4144]

4101. J 
---
[4102, 4145]

4102. Vi
---
[4103, 4146]

4103. Vi
---
[4104, 4147]

4104. J 
---
[4105, 4148]

4105. An
---
[4106, 4149]

4106. Ci
---
[4107, 4150]

4107. Hy
---
[4108, 4151]

4108. J 
---
[4109, 4152]

4109. Ho
---
[4110, 4153]

4110. Ph
---
[4111, 4154]

4111. In
---
[

[4383, 4433]

4383. Ar
---
[4384, 4435]

4384. Co
---
[4385, 4436]

4385. Ac
---
[4386, 4437]

4386. J 
---
[4387, 4438]

4387. Ar
---
[4388, 4439]

4388. Fo
---
[4389, 4440]

4389. La
---
[4390, 4441]

4390. Fo
---
[4391, 4442]

4391. Ad
---
[4392, 4443]

4392. Ch
---
[4393, 4444]

4393. Ex
---
[4394, 4445]

4394. Bi
---
[4395, 4446]

4395. Ne
---
[4396, 4447]

4396. Ri
---
[4397, 4448]

4397. La
---
[4398, 4449]

4398. J 
---
[4399, 4450]

4399. Hi
---
[4400, 4451]

4400. J 
---
[4401, 4452]

4401. Ex
---
[4402, 4453]

4402. Eu
---
[4403, 4454]

4403. J 
---
[4404, 4455]

4404. J 
---
[4405, 4456]

4405. Ac
---
[4406, 4457]

4406. Bi
---
[4407, 4458]

4407. Bi
---
[4408, 4459]

4408. Co
---
[4409, 4460]

4409. Co
---
[4410, 4461]

4410. J 
---
[4411, 4462]

4411. Am
---
[4412, 4463]

4412. Bi
---
[4413, 4464]

4413. Am
---
[4414, 4465]

4414. Z 
---
[4415, 4466]

4415. Ho
---
[4416, 4467]

4416. J 
---
[4417, 4468]

4417. To
---
[4418, 4469]

4418. Br
---
[4419, 4470]

4419. Ph
---
[

[4687, 4739]

4687. Am
---
[4688, 4740]

4688. Am
---
[4689, 4741]

4689. Ad
---
[4690, 4742]

4690. Pr
---
[4691, 4743]

4691. Fo
---
[4692, 4744]

4692. Ps
---
[4693, 4745]

4693. Fo
---
[4694, 4746]

4694. Ne
---
[4695, 4747]

4695. J 
---
[4696, 4748]

4696. Im
---
[4697, 4749]

4697. J 
---
[4698, 4750]

4698. Cl
---
[4699, 4751]

4699. An
---
[4700, 4752]

4700. Co
---
[4701, 4753]

4701. Co
---
[4702, 4754]

4702. An
---
[4703, 4755]

4703. J 
---
[4704, 4756]

4704. J 
---
[4705, 4757]

4705. Bi
---
[4706, 4758]

4706. J 
---
[4707, 4759]

4707. J 
---
[4708, 4760]

4708. Re
---
[4709, 4761]

4709. Be
---
[4710, 4762]

4710. La
---
[4711, 4763]

4711. La
---
[4712, 4764]

4712. La
---
[4713, 4765]

4713. In
---
[4714, 4766]

4714. Ar
---
[4715, 4767]

4715. J 
---
[4716, 4768]

4716. J 
---
[4717, 4769]

4717. Ps
---
[4718, 4770]

4718. J 
---
[4719, 4771]

4719. La
---
[4720, 4772]

4720. Ph
---
[4721, 4773]

4721. Ph
---
[4722, 4774]

4722. Re
---
[4723, 4775]

4723. Ac
---
[

[4995, 5053]

4995. Bi
---
[4996, 5054]

4996. Bi
---
[4997, 5055]

4997. La
---
[4998, 5056]

4998. Cy
---
[4999, 5057]

4999. Ac
---
[5000, 5058]

5000. Sc
---
[5001, 5059]

5001. Re
---
[5002, 5060]

5002. Cy
---
[5003, 5061]

5003. En
---
[5004, 5062]

5004. Pr
---
[5005, 5063]

5005. Bi
---
[5006, 5064]

5006. Sc
---
[5007, 5065]

5007. Am
---
[5008, 5066]

5008. Bi
---
[5009, 5067]

5009. J 
---
[5010, 5068]

5010. J 
---
[5011, 5069]

5011. Am
---
[5012, 5070]

5012. J 
---
[5013, 5071]

5013. J 
---
[5014, 5072]

5014. J 
---
[5015, 5073]

5015. Am
---
[5016, 5074]

5016. Or
---
[5017, 5075]

5017. AM
---
[5018, 5076]

5018. Ho
---
[5019, 5077]

5019. Zh
---
[5020, 5078]

5020. J 
---
[5021, 5079]

5021. J 
---
[5022, 5080]

5022. Am
---
[5023, 5081]

5023. J 
---


In [150]:
k

5022

In [152]:
resNum

5023

In [156]:
strAnalysis=resDg.text[blockST[pre_i]:blockED[i]]
fout_name=os.path.join(fname_tag,str(k+1)+fext)
with open(fout_name, "w", encoding="utf-8") as fid:
    fid.write(strAnalysis)

k=k+1
print([k, i])
print(strAnalysis[1:10])
print("---")


IndexError: list index out of range

In [166]:
blockED[-2:]

[9634653, 9635742]

In [170]:
pre_i

5080

In [171]:
5080

5080

In [176]:
blockED[5082-1]

9635742

In [173]:
i

5082

In [174]:
len(blockED)

5082